#  Install package

In [1]:
# ! pip install pyvis
# ! pip install networkx
# ! pip install dash
# ! pip install altair
# ! pip install panel

# Import package

In [2]:
import os
import requests
import networkx as nx
import pandas as pd
import geopandas as gpd
from pyvis.network import Network
import openai
from IPython.display import display, HTML, Code
from IPython.display import clear_output


# Define Solution class
Please run the following cell to define the functions

In [3]:
# %load_ext autoreload
# %autoreload 2

import LLM_Geo_Constants as constants
import helper
# import LLM_Geo_kernel.Solution as Solution

from LLM_Geo_kernel import Solution

%load_ext autoreload
%autoreload 2

# Demonstration Cases

## Input task and data desciption

In [4]:
# Case 1: population living near hazardous wastes

# task_name ='Resident_at_risk_counting'

# TASK = r"""1) Find out the total population that lives within a tract that contain hazardous waste facilities. The study area is North Carolina, US.
# 2) Generate a map to show the spatial distribution of population at the tract level and highlight the borders of tracts that have hazardous waste facilities.
# """

# DATA_LOCATIONS = ["NC hazardous waste facility ESRI shape file location: https://github.com/gladcolor/LLM- Geo/raw/master/overlay_analysis/Hazardous_Waste_Sites.zip.",
#                   "NC tract boundary shapefile location: https://github.com/gladcolor/LLM-Geo/raw/master/overlay_analysis/tract_shp_37.zip. The tract id column is 'Tract'.",
#                   "NC tract population CSV file location: https://github.com/gladcolor/LLM-Geo/raw/master/overlay_analysis/NC_tract_population.csv. The population is stored in 'TotalPopulation' column. The tract ID column is 'GEOID'."
#                  ]


# Case 2: France_mobility_changes_2020
task_name ='France_mobility_changes_2020'
TASK = r'''
1) Show the 2020 human mobility monthly change rates of each administrative regions in a France choropleth map. Each month is a sub-map in a map matrix，12 months in total. All monthly maps need to use the same colorbar range. The base of the change rate is January 2020. 
2) Draw a line chart to show the monthly change rate trends of all administrative regeions. Each region is a line (the region name is the legend), the x-axis is 2020 months.
'''

DATA_LOCATIONS = ["ESRI shapefile for France administrative regions:" + \
                  "https://github.com/gladcolor/LLM-Geo/raw/master/REST_API/France.zip. " + \
                  "The 'GID_1' column is the administrative region code, 'NAME_1' column is the administrative region name.",
                  "REST API url with parameters for human mobility data access:" + \
                  "http://gis.cas.sc.edu/GeoAnalytics/REST?operation=get_daily_movement_for_all_places&source=twitter&scale=world_first_level_admin&begin=01/01/2020&end=12/31/2020." + \
                  "The response is in CSV format. There are three columns in the response: " + \
                  "place,date (format:2020-01-07), and intra_movement. 'place' column is the administractive region code of every country; codes for France administrative regions start with 'FRA'.",
                 ]

  
# Case 3: COVID-19 prevalence trend
# task_name ='COVID_death_rate'
# TASK = r'''1) Draw a map to show the death rate (death/case) of COVID-19 among the countiguous US counties. Use the accumulated COVID-19 data of 2020.12.31 to compute the death rate. Use scheme ='quantiles' when plotting the map.  Set map projection to 'Conus Albers'. Set map size to 15*10 inches.  
# 2) Draw a scatter plot to show the correlation and trend line of the death rate with the senior resident rate, including the r-square and p-value. Set data point transparency to 50%, regression line as red.  Set figure size to 15*10 inches.  
# '''

# DATA_LOCATIONS = [
#                   r"COVID-19 data case in 2020 (county-level): https://github.com/nytimes/covid-19-data/raw/master/us-counties-2020.csv. This data is for daily accumulated COVID cases and deaths for each county in the US. There are 5 columns: date (format: 2021-02-01), county, state, fips, cases, deaths. ",   
#                   r"Contiguous US county boundary (ESRI shapefile): https://github.com/gladcolor/spatial_data/raw/master/contiguous_counties.zip. The county FIPS column is 'GEOID'. ",
#                   r"Census data (ACS2020): https://raw.githubusercontent.com/gladcolor/spatial_data/master/Demography/ACS2020_5year_county.csv. THe needed columns are: 'FIPS', 'Total Population', 'Total Population: 65 to 74 Years', 'Total Population: 75 to 84 Years', 'Total Population: 85 Years and Over'. Drop rows with NaN cells after loading the used columns.",
#                  ]



# # Case 4: Hospital_accessibility
# task_name ='Hospital_accessibility'
# TASK = r'''
# For each zipcode area in South Carolina (SC), calculate the distance from the centroid of the zipcode area to its nearest hospital, and then create a choropleth distance map of zipcode area polygons (unit: km), also show the hospital.
# '''

# DATA_LOCATIONS = [
# r"SC zipcode boundary shapefile: https://github.com/GIBDUSC/test/raw/master/sc_zip_boundary.zip, the map projection is WGS1984.",
# r"SC hospitals:  https://github.com/gladcolor/spatial_data/raw/master/South_Carolina/SC_hospitals_with_emergency_room_cleaned.csv, location columns: longitude in 'POINT_X' column, latitude in 'POINT_Y' column.",          
# ]


## Case 5: Census API
task_name ='Census_API'
TASK = r''' Show the spatial distribution of the county level median income in the contigous US. Set figure size to (25,15)
'''

DATA_LOCATIONS = [
'''Census API key: ae7be70727932dd6aed257692de3f344365d0678'''              
]



  
save_dir = os.path.join(os.getcwd(), task_name)
os.makedirs(save_dir, exist_ok=True)

# create graph
# model=r"gpt-3.5-turbo"
model=r"gpt-4"
solution = Solution(
                    task=TASK,
                    task_name=task_name,
                    save_dir=save_dir,
                    data_locations=DATA_LOCATIONS,
                    model=model,
                    )
print("Prompt to get solution graph:\n")
# print(solution.graph_prompt)
print(solution.chat_history)

Prompt to get solution graph:

[{'role': 'system', 'content': 'A professional Geo-information scientist and developer good at Python.'}]


## Get graph code from GPT API

In [5]:
print(solution.chat_history)

[{'role': 'system', 'content': 'A professional Geo-information scientist and developer good at Python.'}]


In [6]:
response_for_graph = solution.get_LLM_response_for_graph() 
# print(solution.chat_history)
solution.graph_response = response_for_graph
solution.save_solution()

clear_output(wait=True)
display(Code(solution.code_for_graph, language='python'))

import networkx as nx
G = nx.DiGraph()

# 1 Load Census API key
G.add_node("census_api_key", node_type="data", data_path="", description="Census API key")
G.add_node("set_census_api_key", node_type="operation", description="Set Census API key")
G.add_edge("census_api_key", "set_census_api_key")

# 2 Load county boundary shapefile or URL
G.add_node("county_boundaries_shp", node_type="data", data_path="https://domain.com/county_boundary.shp", description="County boundary shapefile")
G.add_node("load_county_boundaries_shp", node_type="operation", description="Load county boundary shapefile")
G.add_edge("county_boundaries_shp", "load_county_boundaries_shp")

# 3 Create GeoDataFrame for county boundaries
G.add_node("county_boundaries_gdf", node_type="data", description="County boundaries GeoDataFrame")
G.add_edge("load_county_boundaries_shp", "county_boundaries_gdf")

# 4 Fetch median income data using Census API
G.add_node("fetch_median_income_data", node_type="operation", description="Fetch median income data using Census API")
G.add_edge("set_census_api_key", "fetch_median_income_data")

# 5 Store fetched median income dataset
G.add_node("median_income_data", node_type="data", description="Median income dataset")
G.add_edge("fetch_median_income_data", "median_income_data")

# 6 Join median income data with county boundaries GeoDataFrame using a common attribute (e.g., FIPS code)
G.add_node("join_median_income_with_counties", node_type="operation", description="Join median income data with county boundaries GeoDataFrame")
G.add_edge("median_income_data", "join_median_income_with_counties")
G.add_edge("county_boundaries_gdf", "join_median_income_with_counties")

# 7 Store merged GeoDataFrame with income data
G.add_node("merged_counties_income_gdf", node_type="data", description="Merged GeoDataFrame with income data")
G.add_edge("join_median_income_with_counties", "merged_counties_income_gdf")

# 8 Create a choropleth map for spatial distribution of median income
G.add_node("create_choropleth_map", node_type="operation", description="Create choropleth map of median income distribution")
G.add_edge("merged_counties_income_gdf", "create_choropleth_map")

# 9 Set figure size for choropleth map
G.add_node("set_figsize", node_type="operation", description="Set figure size for choropleth map")
G.add_edge("create_choropleth_map", "set_figsize")

# Save the graph to GraphML
nx.write_graphml(G, "E:/Research/LLM-Geo/Census_API/Census_API.graphml")

## Execute code to generate the solution graph

In [7]:
exec(solution.code_for_graph)
solution_graph = solution.load_graph_file()

# Show the graph
G = nx.read_graphml(solution.graph_file)  
nt = helper.show_graph(G)
html_name = os.path.join(os.getcwd(), solution.task_name + '.html')  
# HTML file should in the same directory. See:
# https://stackoverflow.com/questions/65564916/error-displaying-pyvis-html-inside-jupyter-lab-cell
nt.show(name=html_name)
# html_name

E:\Research\LLM-Geo\Census_API.html


## Generate prompts and code for operations (functions)

In [8]:
operations = solution.get_LLM_responses_for_operations(review=False)
solution.save_solution()

all_operation_code_str = '\n'.join([operation['operation_code'] for operation in operations])

# clear_output(wait=True)
# display(Code(all_operation_code_str, language='python'))

1 / 6, LLM is generating code for operation node: set_census_api_key
```python
def set_census_api_key(census_api_key):
    # Description: Set Census API key
    # census_api_key: Census API key
    import censusdata
    censusdata.census.setapikey(census_api_key)
    return fetch_median_income_data
```


2 / 6, LLM is generating code for operation node: load_county_boundaries_shp
```python
import geopandas as gpd

def load_county_boundaries_shp(county_boundaries_shp='https://domain.com/county_boundary.shp'):
    # Description: Load county boundary shapefile
    # county_boundaries_shp: County boundary shapefile URL
    county_boundaries_gdf = gpd.read_file(county_boundaries_shp)
    return county_boundaries_gdf
```



3 / 6, LLM is generating code for operation node: fetch_median_income_data
```python
import pandas as pd
import censusdata

def fetch_median_income_data(set_census_api_key):
    # Description: Fetch median income data using Census API
    # set_census_api_key: Set Census 

## Generate prompts and code for assembly program

In [ ]:
assembly_LLM_response = solution.get_LLM_assembly_response(review=False)
solution.assembly_LLM_response = assembly_LLM_response
solution.save_solution()

# clear_output(wait=True)
# display(Code(solution.code_for_assembly, language='python'))

## Execute assembly code

In [ ]:
all_code = all_operation_code_str + '\n' + solution.code_for_assembly

# display(Code(all_code, language='python'))

all_code = solution.execute_complete_program(code=all_code, try_cnt=10)

